# Desafío 01

## Integrantes

- Acevedo Zain, Gaspar (acevedo.zain.gaspar@gmail.com)

## Consignas

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


## Imports y carga de datos

In [1]:
%pip install numpy scikit-learn

In [79]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

from sklearn.datasets import fetch_20newsgroups
import numpy as np

from sklearn.utils import Bunch
from scipy.sparse import csr_matrix
from typing import Tuple

import random

In [4]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Resolución ejercicio 1

***Enunciado***

- Vectorizar documentos.
- Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
- Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

Comenzamos inicializando un vectorizador del tipo [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) con los parámetros por defecto.

In [11]:
tfidVectorizer_01 = TfidfVectorizer()

Una vez inicializado el vectorizador, se procede a vectorizar el set de entrenamiento (`newsgroups_train`).

In [12]:
X_train = tfidVectorizer_01.fit_transform(newsgroups_train.data)

Se muestra que hay $11314$ documentos vectorizados en el set de entrenamiento.

In [16]:
cant_docs = X_train.shape[0]
print(f"Cantidad de documentos en train: {cant_docs}")

Cantidad de documentos en train: 11314


Se muestra que el tamaño del vocabulario en el set de entrenamiento es de $101631$.

In [137]:
tam_vocab = X_train.shape[1]
print(f"Tamaño del vocabulario en train: {tam_vocab}")

Tamaño del vocabulario en train: 101631


Se crea a continuación un diccionario que va de índices a términos (código dado por la materia).

In [138]:
idx2word = {v: k for k,v in tfidVectorizer_01.vocabulary_.items()}

A continuación, se seleccionan al azar $5$ documentos vectorizados del set de entrenamiento.

In [17]:
random_idxs = [random.randint(0, cant_docs -1) for _ in range(5)]

In [48]:
corpus = newsgroups_train

### Funciones de utilidad

Se definen las siguientes funciones de utilidad:

- `get_documents_similarity`, la cual, dados una `matriz documento-término` y un índice de un documento de la misma, obtiene la `similitud coseno` entre dicho documento y el resto de los documentos de la matriz, como así también los índices de estos documentos.
- `get_k_most_similar_documents`, la cual utiliza la función anterior, y devuelve a los $K$ documentos con mayor `similitud coseno` junto a sus índices. Por defecto $K = 5$.
- `imprimir_informe`, la cual devuelve un informe de un documento de un corpus, al cual se compara utilizando la similitud coseno con otros documentos del corpus.
- `get_common_words`, la cual devuelve los términos que tienen en común dos documentos vectorizados.

In [72]:
def get_documents_similarity(matriz_dt: csr_matrix, idx: int) -> Tuple[np.ndarray, np.ndarray]:
  """
  Devuelve la similitud coseno de un documento respecto al resto de los
    documentos de una matriz documento-término.

  Args:
    matriz_dt (csr_matrix): Matrix documento-término a partir de la cual se
      obtendrá la similitud coseno de un documento, dado su índice.
    idx (int): índice del documento sobre el cual se trabajará.
  Returns:
    Tuple[np.ndarray, np.ndarray]:
      Elem 0: documentos ordenados de mayor a menor similitud coseno.
      Elem 1: argumentos correspondientes a Elem 0.
  Raises:
    IndexError, si el parámetro `idx` está fuera del rando de índices de
      `matriz_dt`.
  """

  if idx >= matriz_dt.shape[0]:
    raise IndexError(f"El parámetro `idx` está fuera del rango del corpus \
    ({matriz_dt.shape[0]}).")

  documents_similarity = cosine_similarity(matriz_dt[idx], matriz_dt)[0]

  sorted_docs = np.sort(documents_similarity)[::-1]
  arg_docs = np.argsort(documents_similarity)[::-1]
  return sorted_docs, arg_docs


In [73]:
def get_k_most_similar_documents(matriz_dt: csr_matrix, idx: int, K: int = 5) -> Tuple[np.ndarray, np.ndarray]:
  """
  Dado una matríz documento-término y un índice correspondiente a uno de sus documentos, devuelve
  los K documentos con mayor similitud coseno respecto del documento indicado.

  Args:
    matriz_dt (csr_matrix): Matrix documento-término a partir de la cual se
      obtendrá la similitud coseno de un documento, dado su índice.
    idx (int): índice del documento sobre el cual se trabajará.
    K (int): número de documentos con mayor similitud que se quiere encontrar.
      Por defecto es 5.
  Returns:
    np.ndarray: array de longitud K que contiene la similitud coseno del
      documento a analizar respecto al resto, ordenados de mayor a menor.
    Tuple[np.ndarray, np.ndarray]:
      Elem 0: `K` documentos ordenados de mayor a menor similitud coseno.
      Elem 1: argumentos correspondientes a Elem 0.
  Raises:
    IndexError, si el parámetro `idx` está fuera del rando de índices de
      `matriz_dt`.
  """

  if idx >= matriz_dt.shape[0]:
    raise IndexError(f"El parámetro `idx` está fuera del rango de la matriz \
    documento-término ({matriz_dt.shape[0]}).")

  docs_similarity, docs_args  = get_documents_similarity(matriz_dt=matriz_dt, idx=idx)

  # El índice 0 corresponde al propio documento
  return docs_similarity[1:K + 1], docs_args[1:K + 1]

In [111]:
def imprimir_informe(corpus: Bunch, idx: int, similar_docs: np.ndarray, similar_docs_idxs: np.ndarray) -> str:
  """
  Esta función permite mostrar un informe correspondiente al documento de un
  corpus dado, en donde se muestran los K documentos con mayor similitud coseno.

  Args:
    corpus (Bunch): corpus sobre el cual se trabaja.
    idx (int): índice del documento sobre el cual se hace el análisis.
    similar_docs (np.ndarray): array que contiene los K documentos con mayor
      similitud coseno respecto del documento de análisis.
    similar_docs_idxs (np.ndarray): array que contiene los índices correspondientes
      a los documentos de `similar_docs`.

  Returns:
    string: Informe, con tabla comparativa de documentos.
  """
  doc_data = corpus.data[idx]
  doc_target_idx = corpus.target[idx]
  doc_type = corpus.target_names[doc_target_idx]

  informe = f"En esta sección se realiza el análisis del documento con índice ${idx}$\n\n"
  informe += f"Este es un documento del tipo `{doc_type}`.\n\n"
  informe += f"A continuación, se muestran los  5  documentos con mayor similitud coseno, dentro del corpus de ***entrenamiento***.\n\n"

  tabla_comparativa = f"| Índice documento | Tipo de documento | Similitud coseno con documento ${idx}$ | Coinciden los tipos de documento? |\n"
  tabla_comparativa += f"| --- | --- | --- | --- |\n"

  for i in range(len(similar_docs)):
    # Similitud coseno entre documentos
    similarity = similar_docs[i]
    # Índice dentro del corpus del documento con el cual se comparó
    similarity_idx = similar_docs_idxs[i]
    # Índice del tipo de documento contra el cual se comparó
    similarity_target_idx = corpus.target[similarity_idx]
    # Tipo de documento contra el cual se comparó
    similarity_type = corpus.target_names[similarity_target_idx]

    same_type = "Si" if doc_type == similarity_type else "No"

    tabla_comparativa += f"| ${similarity_idx}$ | {similarity_type} | ${np.round(similarity, 4)}$ | {same_type} |\n"

  return informe + tabla_comparativa

In [142]:
def get_common_words(vectorized_corpus: csr_matrix, idx1: int, idx2: int, idx2word: dict) -> list:
  """
  Devuelve los términos comunes de dos documentos vectorizados.

  Args:
    vectorized_corpus (csr_matrix): Corpus (vectorizado) al que corresponden
      los documentos.
    idx1 (int): índice dentro del vectorized corpus del primer documento.
    idx2 (int): índice dentro del vectorized corpus del segundo documento.
    idx2word: diccionario que va de índices a términos del corpus.

  Returns:
    list: lista con términos en común.
  """
  # Términos del documento 1 (o documento base)
  _, cols1 = (vectorized_corpus[idx1]).nonzero()
  # Términos del documento 2
  _, cols2 = (vectorized_corpus[idx2]).nonzero()

  # Los trato como `sets` para facilitar la `intersección`
  cols1set = set(cols1)
  cols2set = set(cols2)
  common_cols = list(cols1set.intersection(cols2set))

  common_words = []

  for col in common_cols:
    common_words.append(idx2word[col])

  return common_words

### Análisis del documento con índice $11097$

En esta sección se realiza el análisis del documento con índice $11097$

Este es un documento del tipo `soc.religion.christian`.

A continuación, se muestran los  5  documentos con mayor similitud coseno, dentro del corpus de ***entrenamiento***.

| Índice documento | Tipo de documento | Similitud coseno con documento $11097$ | Coinciden los tipos de documento? |
| --- | --- | --- | --- |
| $4626$ | soc.religion.christian | $0.2651$ | Si |
| $11117$ | soc.religion.christian | $0.2036$ | Si |
| $7358$ | talk.religion.misc | $0.2027$ | No |
| $4800$ | soc.religion.christian | $0.2022$ | Si |
| $8807$ | talk.religion.misc | $0.1759$ | No |

In [112]:
idx = random_idxs[0]
similar_docs, similar_docs_idx = get_k_most_similar_documents(matriz_dt=X_train, idx=idx)
informe = imprimir_informe(corpus=corpus, idx=idx, similar_docs=similar_docs, similar_docs_idxs=similar_docs_idx)

In [113]:
print(informe)

En esta sección se realiza el análisis del documento con índice $11097$

Este es un documento del tipo `soc.religion.christian`.

A continuación, se muestran los  5  documentos con mayor similitud coseno, dentro del corpus de ***entrenamiento***.

| Índice documento | Tipo de documento | Similitud coseno con documento $11097$ | Coinciden los tipos de documento? |
| --- | --- | --- | --- |
| $4626$ | soc.religion.christian | $0.2651$ | Si |
| $11117$ | soc.religion.christian | $0.2036$ | Si |
| $7358$ | talk.religion.misc | $0.2027$ | No |
| $4800$ | soc.religion.christian | $0.2022$ | Si |
| $8807$ | talk.religion.misc | $0.1759$ | No |



Como se puede observar, para el primer documento analizado, el cual tiene el índice $11097$, de los $5$ documentos con mayor similitud coseno se encontró que $3$ de ellos tienen el mismo tipo (`soc.religion.christian`), mientras que el resto no, aunque son de una *temática* similar (`talk.religion.misc`).

A continuación se muestra el contenido de todos los documentos utilizados en esta comparación.

Contenido del documento analizado/documento base (índice $11097$)

- Tipo de documento: `soc.religion.christian`.

Como se puede observar, aparecen *términos* como **God**, **Heaven**, **mercy**, lo cual dan a entender que es un documento que referencia a la fe Cristiana (de ahí su tipo).

In [115]:
print(corpus.data[idx])


I have also heard it called an expression of mercy, because Heaven would be
far more agonizing for those who had rejected God.



Contenido del documento con índice $4626$:
- Similitud coseno: $0.2651$.
- Tipo de documento: `soc.religion.christian`.
- Tiene el mismo tipo que el documento base?: ***Si***.

En este documento aparecen los términos **God**, **mercy** y **Hell**, el cual da a enteder que es un documento que referencia a la fe Cristiana (tipo `soc.religion.christian`).

Los términos en común con el documento base son: `god`, `be`, `an`, `expression`, `mercy`, `of`, `because`.

In [116]:
print(corpus.data[similar_docs_idx[0]])

Quoth the Moderator:


In a short poem ("God in His mercy made / the fixed pains of Hell"),
C. S. Lewis expresses an idea that I'm sure was current among others,
but I haven't be able to find its source:

that even Hell is an expression of mercy, because God limits the amount
of separation from Him, and hence the amount of agony, that one can
achieve.



In [147]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[0], idx2word=idx2word)
print(common_words)


['god', 'be', 'an', 'expression', 'mercy', 'of', 'because']


## Resolución ejercicio 2

## Resolución ejercicio 3

## Resolución ejercicio 4